Please run those two cells before running the Notebook!

As those plotting settings are standard throughout the book, we do not show them in the book every time we plot something.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
# FIX: Use the official public API path from pandas.errors
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# feel free to modify, for example, change the context to "notebook"
sns.set_theme(context="talk", style="whitegrid", 
              palette="colorblind", color_codes=True, 
              rc={"figure.figsize": [12, 8]})

# Chapter 7 - ML-based Approaches to Time Series Forecasting

## 7.1 Validation methods for time series

### How to do it...

1. Import the libraries and authenticate:

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
import nasdaqdatalink

nasdaqdatalink.ApiConfig.api_key = "YK5hwjxe3Swf2y_zTjxx"

2. Download the monthly US unemployment rate from years 2010-2019:

In [ ]:
# FIX: Install the pandas-datareader library
!pip install pandas-datareader

import pandas_datareader.data as web
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# FIX: Download the data directly from FRED using its series ID "UNRATENSA"
df = (
    web.DataReader(name="UNRATENSA", 
                   data_source="fred",
                   start="2010-01-01",
                   end="2019-12-31")
    .rename(columns={"UNRATENSA": "unemp_rate"})
)

df.plot(title="Unemployment rate (US) - monthly")

plt.tight_layout()
sns.despine()

3. Create simple features:

In [ ]:
df["linear_trend"] = range(len(df))
df["month"] = df.index.month
df

4. Use one-hot encoding for the month feature:

In [ ]:
month_dummies = pd.get_dummies(
    df["month"], drop_first=True, prefix="month"
)

df = df.join(month_dummies) \
       .drop(columns=["month"])

df

5. Separate the target from the features:

In [ ]:
X = df.copy()
y = X.pop("unemp_rate")

In [ ]:
X.shape

6. Define the expanding window walk-forward validation and print the indices of the folds:

In [ ]:
expanding_cv = TimeSeriesSplit(n_splits=5, test_size=12)

for fold, (train_ind, valid_ind) in enumerate(expanding_cv.split(X)):
    print(f"Fold {fold} ----")
    print(f"Train indices: {train_ind}")
    print(f"Valid indices: {valid_ind}")


7. Evaluate the model's performance using the expanding window validation:

In [ ]:
scores = [] 

for train_ind, valid_ind in expanding_cv.split(X):
    lr = LinearRegression()
    lr.fit(X.iloc[train_ind], y.iloc[train_ind])
    y_pred = lr.predict(X.iloc[valid_ind])
    scores.append(
        mean_absolute_percentage_error(y.iloc[valid_ind], y_pred)
    )

print(f"Scores: {scores}")
print(f"Avg. score: {np.mean(scores)}")

In [ ]:
cv_scores = cross_validate(
    LinearRegression(), 
    X, y, 
    cv=expanding_cv, 
    scoring=["neg_mean_absolute_percentage_error", 
             "neg_root_mean_squared_error"]
)
pd.DataFrame(cv_scores)

8. Define the sliding window validation and print the indices of the folds:

In [ ]:
sliding_cv = TimeSeriesSplit(
    n_splits=5, test_size=12, max_train_size=60
)

for fold, (train_ind, valid_ind) in enumerate(sliding_cv.split(X)):
    print(f"Fold {fold} ----")
    print(f"Train indices: {train_ind}")
    print(f"Valid indices: {valid_ind}")


9. Evaluate the model's performance using the sliding window validation:

In [ ]:
cv_scores = cross_validate(
    LinearRegression(), 
    X, y, 
    cv=sliding_cv, 
    scoring=["neg_mean_absolute_percentage_error", 
             "neg_root_mean_squared_error"]
)
pd.DataFrame(cv_scores)

In [ ]:
-1 * cv_scores["test_neg_mean_absolute_percentage_error"].mean()

## 7.2 Feature engineering for time series

### How to do it...

1. Import the libraries:

In [ ]:
# FIX: Install the scikit-lego library
!pip install scikit-lego

In [ ]:
import numpy as np
import pandas as pd
from datetime import date

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklego.preprocessing import RepeatingBasisFunction

In [ ]:
# temporary change for this recipe
sns.set_palette(["grey", "blue"])

2. Generate a time series with repeating patterns:

In [ ]:
np.random.seed(42)

range_of_dates = pd.date_range(start="2017-01-01", 
                               end="2019-12-31")
X = pd.DataFrame(index=range_of_dates)

X["day_nr"] = range(len(X))
X["day_of_year"] = X.index.day_of_year

signal_1 = 2 + 3 * np.sin(X["day_nr"] / 365 * 2 * np.pi)
signal_2 = 2 * np.sin(X["day_nr"] / 365 * 4 * np.pi + 365/2)
noise = np.random.normal(0, 0.81, len(X))

y = signal_1 + signal_2 + noise
y.name = "y"

y.plot(title="Generated time series")

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_10")

In [ ]:
X

3. Store the time series in a new DataFrame:

In [ ]:
results_df = y.to_frame()
results_df.columns = ["y_true"]
results_df.head()

4. Encode the month information as dummies:

In [ ]:
X_1 = pd.get_dummies(
    X.index.month, drop_first=True, prefix="month"
)
X_1.index = X.index
X_1

5. Fit a linear regression model and plot the in-sample prediction:

In [ ]:
model_1 = LinearRegression().fit(X_1, y)

results_df["y_pred_1"] = model_1.predict(X_1)
(
    results_df[["y_true", "y_pred_1"]]
    .plot(title="Fit using month dummies")
)

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_12")

6. Define functions used for creating the cyclical encoding:

In [ ]:
def sin_transformer(period):
	return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))

def cos_transformer(period):
	return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

7. Encode the month and day information using cyclical encoding:

In [ ]:
X_2 = X.copy()
X_2["month"] = X_2.index.month

X_2["month_sin"] = sin_transformer(12).fit_transform(X_2)["month"]
X_2["month_cos"] = cos_transformer(12).fit_transform(X_2)["month"]

X_2["day_sin"] = (
    sin_transformer(365).fit_transform(X_2)["day_of_year"]
)
X_2["day_cos"] = (
    cos_transformer(365).fit_transform(X_2)["day_of_year"]
)

fig, ax = plt.subplots(2, 1, sharex=True, figsize=(16,8))
X_2[["month_sin", "month_cos"]].plot(ax=ax[0])
ax[0].legend(loc="center left", bbox_to_anchor=(1, 0.5))
X_2[["day_sin", "day_cos"]].plot(ax=ax[1])
ax[1].legend(loc="center left", bbox_to_anchor=(1, 0.5))
plt.suptitle("Cyclical encoding with sine/cosine transformation")

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_13")

In [ ]:
(
    X_2[X_2.index.year == 2017]
    .plot(
        kind="scatter", 
        x="month_sin", 
        y="month_cos", 
        figsize=(8, 8),
        title="Cyclical encoding using sine/cosine transformations"
    )
)

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_14")

8. Fit a model using the daily sine/cosine features:

In [ ]:
X_2 = X_2[["day_sin", "day_cos"]]

model_2 = LinearRegression().fit(X_2, y)

results_df["y_pred_2"] = model_2.predict(X_2)
(
    results_df[["y_true", "y_pred_2"]]
    .plot(title="Fit using sine/cosine features")
)

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_15")

9. Create features using the radial basis functions:

In [ ]:
rbf = RepeatingBasisFunction(n_periods=12,
                             column="day_of_year",
                             input_range=(1,365),
                             remainder="drop")
rbf.fit(X)
X_3 = pd.DataFrame(index=X.index, 
                   data=rbf.transform(X))

In [ ]:
X_3.plot(subplots=True, sharex=True, 
         title="Radial Basis Functions", 
         legend=False, figsize=(14, 10))

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_16")

10. Fit a model using the RBF features:

In [ ]:
model_3 = LinearRegression().fit(X_3, y)

results_df["y_pred_3"] = model_3.predict(X_3)
(
    results_df[["y_true", "y_pred_3"]]
    .plot(title="Fit using RBF features")
)

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_17")

### There's more

1. Import the libraries:

In [ ]:
# FIX: Install the tsfresh library
!pip install tsfresh

In [ ]:
from sktime.transformations.series.date import DateTimeFeatures

from tsfresh import extract_features
from tsfresh.feature_extraction import settings
from tsfresh.utilities.dataframe_functions import roll_time_series

2. Extract the datetime-features using `sktime`:

In [ ]:
dt_features = DateTimeFeatures(
    ts_freq="D", feature_scope="comprehensive"
)
features_df_1 = dt_features.fit_transform(y)
features_df_1.head()

3. Prepare the dataset for feature extraction with `tsfresh`:

In [ ]:
df = y.to_frame().reset_index(drop=False)
df.columns = ["date", "y"]
df["series_id"] = "a"
df

4. Create a rolled-up DataFrame for feature extraction:

In [ ]:
df_rolled = roll_time_series(
    df, column_id="series_id", column_sort="date",
    max_timeshift=30, min_timeshift=7
).drop(columns=["series_id"])
df_rolled

5. Extract the minimal set of features:

In [ ]:
settings_minimal = settings.MinimalFCParameters() 
settings_minimal

In [ ]:
features_df_2 = extract_features(
    df_rolled, column_id="id", 
    column_sort="date", 
    default_fc_parameters=settings_minimal
)

In [ ]:
features_df_2

6. Clean up the index and inspect the features:

In [ ]:
features_df_2 = (
    features_df_2
    .set_index(
        features_df_2.index.map(lambda x: x[1]), drop=True
    )
)
features_df_2.index.name = "last_date"
features_df_2.head(25).round(4)

## 7.3 Time series forecasting as reduced regression

### Getting ready

1. Import the libraries and authenticate:

In [ ]:
import pandas as pd
import numpy as np
import nasdaqdatalink

nasdaqdatalink.ApiConfig.api_key = "YOUR_KEY_HERE"

2. Download the monthly US unemployment rate from years 2010-2019:

In [ ]:
y = (
    nasdaqdatalink.get(dataset="FRED/UNRATENSA", 
                       start_date="2010-01-01", 
                       end_date="2019-12-31")
    .rename(columns={"Value": "unemp_rate"})
)
y.index = y.index.to_period(freq="M")

In [ ]:
y

### How to do it...

1. Import the libraries:

In [ ]:
from sktime.utils.plotting import plot_series
from sktime.forecasting.model_selection import (
    temporal_train_test_split, ExpandingWindowSplitter
)
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import (
    make_reduction, TransformedTargetForecaster, 
    EnsembleForecaster
)
from sktime.performance_metrics.forecasting import (
    mean_absolute_percentage_error
)
from sktime.transformations.series.detrend import (
    Deseasonalizer, Detrender
)
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.forecasting.model_evaluation import evaluate
from sktime.forecasting.arima import AutoARIMA

from sklearn.ensemble import RandomForestRegressor

2. Split the time series into training and tests sets:

In [ ]:
y_train, y_test = temporal_train_test_split(
    y, test_size=12
)
plot_series(
    y_train, y_test, 
    labels=["y_train", "y_test"]
)

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_22")

3. Set the forecast horizon to 12 months:

In [ ]:
fh = ForecastingHorizon(y_test.index, is_relative=False)
fh

4. Instantiate the reduced regression model, fit it to the data and create predictions:

In [ ]:
regressor = RandomForestRegressor(random_state=42)
rf_forecaster = make_reduction(
    estimator=regressor, 
    strategy="recursive", 
    window_length=12
)
rf_forecaster.fit(y_train)
y_pred_1 = rf_forecaster.predict(fh)

5. Evaluate the performance of the forecasts:

In [ ]:
mape_1 = mean_absolute_percentage_error(
    y_test, y_pred_1, symmetric=False
)
fig, ax = plot_series(
    y_train["2016":], y_test, y_pred_1,
    labels=["y_train", "y_test", "y_pred"]
)
ax.set_title(f"MAPE: {100*mape_1:.2f}%")

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_23")

6. Deseasonalize the time series:

In [ ]:
deseasonalizer = Deseasonalizer(model="additive", sp=12)
y_deseas = deseasonalizer.fit_transform(y_train)
plot_series(
    y_train, y_deseas, 
    labels=["y_train", "y_deseas"]
)

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_24")

In [ ]:
plot_series(
    deseasonalizer.seasonal_, 
    labels=["seasonal_component"]
)

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_25")

7. Detrend the time series:

In [ ]:
forecaster = PolynomialTrendForecaster(degree=1)
transformer = Detrender(forecaster=forecaster)
y_detrend = transformer.fit_transform(y_deseas)

# in-sample predictions
forecaster = PolynomialTrendForecaster(degree=1)
y_in_sample = (
    forecaster
    .fit(y_deseas)
    .predict(fh=-np.arange(len(y_deseas)))
)

plot_series(
    y_deseas, y_in_sample, y_detrend, 
    labels=["y_deseas", "linear trend", "resids"]
)

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_26")

8. Combine the components into a pipeline, fit it to the original time series and obtain predictions:

In [ ]:
rf_pipe = TransformedTargetForecaster(
    steps = [
        ("deseasonalize", Deseasonalizer(model="additive", sp=12)),
        ("detrend", Detrender(
            forecaster=PolynomialTrendForecaster(degree=1)
        )),
        ("forecast", rf_forecaster),
    ]
)
rf_pipe.fit(y_train)
y_pred_2 = rf_pipe.predict(fh)

9. Evaluate the pipeline's predictions:

In [ ]:
mape_2 = mean_absolute_percentage_error(
    y_test, y_pred_2, symmetric=False
)
fig, ax = plot_series(
    y_train["2016":], y_test, y_pred_2,
    labels=["y_train", "y_test", "y_pred"]
)
ax.set_title(f"MAPE: {100*mape_2:.2f}%")

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_27")

10. Evaluate the performance using expanding window cross-validation:

In [ ]:
cv = ExpandingWindowSplitter(
    fh=list(range(1,13)), 
    initial_window=12*5, 
    step_length=12
)

cv_df = evaluate(
    forecaster=rf_pipe, 
    y=y, 
    cv=cv, 
    strategy="refit", 
    return_data=True
)

cv_df

In [ ]:
for ind, row in cv_df.iterrows():
    print(f"Fold {ind} ----")
    print(f"Training: {row['y_train'].index.min()} - {row['y_train'].index.max()}")
    print(f"Training: {row['y_test'].index.min()} - {row['y_test'].index.max()}")


11. Plot the predictions from the cross-validation folds:

In [ ]:
n_fold = len(cv_df)

plot_series(
    y, 
    *[cv_df["y_pred"].iloc[x] for x in range(n_fold)],
    markers=["o", *["."] * n_fold],
    labels=["y_true"] + [f"cv: {x}" for x in range(n_fold)]
)

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_29")

12. Create an ensemble forecast using the RF pipeline and AutoARIMA:

In [ ]:
ensemble = EnsembleForecaster(
    forecasters = [
        ("autoarima", AutoARIMA(sp=12)),
        ("rf_pipe", rf_pipe)
    ]
)
ensemble.fit(y_train)
y_pred_3 = ensemble.predict(fh)

13. Evaluate the ensemble's predictions:

In [ ]:
mape_3 = mean_absolute_percentage_error(
    y_test, y_pred_3, symmetric=False
)
fig, ax = plot_series(
    y_train["2016":], y_test, y_pred_3,
    labels=["y_train", "y_test", "y_pred"]
)
ax.set_title(f"MAPE: {100*mape_3:.2f}%")

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_30")

### There's more

1. Create naive forecasts using the `NaiveForecaster`:

In [ ]:
from sktime.forecasting.naive import NaiveForecaster

fh = ForecastingHorizon(y_test.index, is_relative=False)

naive_fcst_last = NaiveForecaster(strategy="last")
naive_fcst_last.fit(y_train)
y_last = naive_fcst_last.predict(fh)

naive_fcst_seas = NaiveForecaster(strategy="last", sp=12)
naive_fcst_seas.fit(y_train)
y_seasonal_last = naive_fcst_seas.predict(fh)

plot_series(
    y_train["2016":], y_test, y_last, y_seasonal_last, 
    labels=["y_train", "y_test", "y_pred_last", "y_pred_last_seas"]
);

2. Inspect all the available models:

In [ ]:
from sktime.registry import all_estimators

all_estimators("forecaster", as_dataframe=True)

## 7.4 Forecasting with Meta's Prophet

### How to do it...

1. Import the libraries and authenticate with Nasdaq Data Link:

In [ ]:
import pandas as pd
import nasdaqdatalink
from prophet import Prophet
from prophet.plot import add_changepoints_to_plot

nasdaqdatalink.ApiConfig.api_key = "YK5hwjxe3Swf2y_zTjxx"

2. Download the daily gold prices:

In [ ]:
df = nasdaqdatalink.get(
    dataset="WGC/GOLD_DAILY_USD",
    start_date="2015-01-01",
    end_date="2019-12-31"
)

df.plot(title="Daily gold prices (2015-2019)")

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_31")

3. Rename the columns:

In [ ]:
df = df.reset_index(drop=False)
df.columns = ["ds", "y"]

4. Split the series into the training and test sets:

In [ ]:
train_indices = df["ds"] < "2019-10-01"
df_train = df.loc[train_indices].dropna()
df_test = (
    df
    .loc[~train_indices]
    .reset_index(drop=True)
)

5. Create the instance of the model and fit it to the data:

In [ ]:
prophet = Prophet(changepoint_range=0.9)
prophet.add_country_holidays(country_name="US")
prophet.add_seasonality(
    name="monthly", period=30.5, fourier_order=5
)
prophet.fit(df_train)

6. Forecast the gold prices for the fourth quarter of 2019 and plot the results:

In [ ]:
df_future = prophet.make_future_dataframe(
    periods=len(df_test), freq="B"
)
df_pred = prophet.predict(df_future)
prophet.plot(df_pred)

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_32")

In [ ]:
df_pred.columns

7. Add changepoints to the plot:

In [ ]:
fig = prophet.plot(df_pred)
a = add_changepoints_to_plot(
    fig.gca(), prophet, df_pred
)

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_33")

In [ ]:
prophet.changepoints

8. Inspect the decomposition of the time series:

In [ ]:
prophet.plot_components(df_pred)

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_34")

9. Merge the test set with the forecasts:

In [ ]:
SELECTED_COLS = [
    "ds", "yhat", "yhat_lower", "yhat_upper"
]

df_pred = (
    df_pred
    .loc[:, SELECTED_COLS]
    .reset_index(drop=True)
)
df_test = df_test.merge(df_pred, on=["ds"], how="left")
df_test["ds"] = pd.to_datetime(df_test["ds"])
df_test = df_test.set_index("ds")

10. Plot the test values vs. predictions:

In [ ]:
fig, ax = plt.subplots(1, 1)

PLOT_COLS = [
    "y", "yhat", "yhat_lower", "yhat_upper"
]
ax = sns.lineplot(data=df_test[PLOT_COLS])
ax.fill_between(
    df_test.index,
    df_test["yhat_lower"],
    df_test["yhat_upper"],
    alpha=0.3
)

ax.set(
    title="Gold Price - actual vs. predicted",
    xlabel="Date",
    ylabel="Gold Price ($)"
)

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_35")

### There's more

1. Import the libraries:

In [ ]:
from prophet.diagnostics import (cross_validation, 
                                 performance_metrics)
from prophet.plot import plot_cross_validation_metric

2. Run Prophet's cross-validation :

In [ ]:
df_cv = cross_validation(
    prophet, 
    initial="756 days", 
    period="60 days", 
    horizon = "60 days"
)

df_cv

In [ ]:
df_cv["cutoff"].value_counts().sort_index()

In [ ]:
df_cv.head(50)

3. Calculate the aggregated performance metrics:

In [ ]:
df_p = performance_metrics(df_cv)
df_p

4. Plot the MAPE score:

In [ ]:
plot_cross_validation_metric(df_cv, metric="mape")

plt.tight_layout()
sns.despine()
# plt.savefig("images/figure_7_38")

## 7.5 AutoML for time series forecasting with PyCaret

### Getting ready

1. Import the libraries and authenticate:

In [ ]:
import nasdaqdatalink

nasdaqdatalink.ApiConfig.api_key = "YOUR_KEY_HERE"

2. Download the monthly US unemployment rate from years 2010-2019:

In [ ]:
df = (
    nasdaqdatalink.get(dataset="FRED/UNRATENSA", 
                       start_date="2010-01-01", 
                       end_date="2019-12-31")
    .rename(columns={"Value": "unemp_rate"})
)
df.plot(title="Unemployment rate (US) - monthly");

### How to do it...

1. Import the libraries:

In [ ]:
from pycaret.datasets import get_data
from pycaret.time_series import TSForecastingExperiment

2. Set up the experiment:

In [ ]:
exp = TSForecastingExperiment()
exp.setup(df, fh=6, fold=5, session_id=42)

3. Explore the time series using visualizations:

In [ ]:
exp.plot_model(
    plot="diagnostics", 
    fig_kwargs={"height": 800, "width": 1000}
)

In [ ]:
exp.plot_model(plot="cv")

Some additional plots:

In [ ]:
exp.plot_model(plot="ts")

In [ ]:
exp.plot_model(plot="acf")

In [ ]:
exp.plot_model(plot="decomp_stl")

In [ ]:
exp.plot_model(plot="periodogram")

In [ ]:
exp.plot_model(plot="fft")

4. Run statistical tests on the time series:

In [ ]:
exp.check_stats()

In [ ]:
exp.check_stats(test="summary")

5. Find the five best fitting pipelines:

In [ ]:
best_pipelines = exp.compare_models(
    sort="MAPE", turbo=False, n_select=5
)

In [ ]:
best_pipelines

In [ ]:
# recover the DataFrame with the metrics
compare_metrics_base = exp.pull()
compare_metrics_base

In [ ]:
# inspect the available models
exp.models()

6. Tune the best pipelines:

In [ ]:
best_pipelines_tuned = [exp.tune_model(model) for model in best_pipelines]
best_pipelines_tuned

7. Blend the 5 tuned pipelines:

In [ ]:
blended_model = exp.blend_models(
    best_pipelines_tuned, method="mean"
)

8. Create the predictions using the blended model and plot the forecasts:

In [ ]:
y_pred = exp.predict_model(blended_model)

In [ ]:
exp.plot_model(estimator=blended_model)

9. Finalize the model:

In [ ]:
final_model = exp.finalize_model(blended_model)
exp.plot_model(final_model)

In [ ]:
y_pred = exp.predict_model(final_model)
print(y_pred)